In [28]:
import os
import sys
import csv
sys.path.insert(0, os.path.abspath('../'))

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax
import arrayfire as af

from dg_maxwell import params
from dg_maxwell import utils
from dg_maxwell import lagrange
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import msh_parser
from dg_maxwell import isoparam

af.set_backend(params.backend)
af.set_device(params.device)
af.info()

In [29]:
nodes, elements = msh_parser.read_order_2_msh('read_and_plot_mesh/mesh/square_10_10.msh')

gmshTranslator: Initializing...
gmshTranslator: Mesh has 441 nodes.
gmshTranslator: Mesh has 144 elements.
gmshTranslator: Processed 589 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements


In [30]:
def u_init(x, y):
    return np.e**(-x**2 / 0.6**2)

In [31]:
N_LGL   = 8
x_e_ij  = af.np_to_af_array(np.zeros([N_LGL * N_LGL, len(elements)]))
y_e_ij  = af.np_to_af_array(np.zeros([N_LGL * N_LGL, len(elements)]))

xi_LGL  = lagrange.LGL_points(N_LGL)
eta_LGL = lagrange.LGL_points(N_LGL)

Xi_LGL, Eta_LGL = utils.af_meshgrid(xi_LGL, eta_LGL)
xi_i  = af.flat(Xi_LGL)
eta_j = af.flat(Eta_LGL)

for element_tag, element in enumerate(elements):
    x_e_ij[:, element_tag] = isoparam.isoparam_x_2D(nodes[element, 0], xi_i, eta_j)
    y_e_ij[:, element_tag] = isoparam.isoparam_y_2D(nodes[element, 1], xi_i, eta_j)


/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [32]:
u_e_ij = u_init(x_e_ij, y_e_ij)

In [33]:
_, Lp_xi  = lagrange.lagrange_polynomials(xi_LGL)
_, Lq_eta = lagrange.lagrange_polynomials(eta_LGL)
Lp_xi = af.np_to_af_array(Lp_xi)
Lq_eta = af.np_to_af_array(Lq_eta)

dLp_dxi_coeffs  = utils.polynomial_derivative(Lp_xi)
dLq_deta_coeffs = utils.polynomial_derivative(Lq_eta)

In [34]:
Lp_xi_tp = af.reorder(Lp_xi, d0 = 2, d1 = 0, d2 = 1)
Lp_xi_tp = af.tile(Lp_xi_tp, d0 = N_LGL)
Lp_xi_tp = af.moddims(Lp_xi_tp, d0 = N_LGL * N_LGL, d1 = 1, d2 = N_LGL)
Lp_xi_tp = af.reorder(Lp_xi_tp, d0 = 0, d1 = 2, d2 = 1)

Lq_eta_tp = af.reorder(Lq_eta, d0 = 0, d1 = 2, d2 = 1)
Lq_eta_tp = af.tile(Lq_eta_tp, d0 = N_LGL)
Lq_eta_tp = af.reorder(Lq_eta_tp, d0 = 0, d1 = 2, d2 = 1)

In [35]:
dLp_dxi_tp = af.reorder(dLp_dxi_coeffs, d0 = 2, d1 = 0, d2 = 1)
dLp_dxi_tp = af.tile(dLp_dxi_tp, d0 = N_LGL)
dLp_dxi_tp = af.moddims(dLp_dxi_tp, d0 = N_LGL * N_LGL, d1 = 1, d2 = N_LGL - 1)
dLp_dxi_tp = af.reorder(dLp_dxi_tp, d0 = 0, d1 = 2, d2 = 1)

dLq_deta_tp = af.reorder(dLq_deta_coeffs, d0 = 0, d1 = 2, d2 = 1)
dLq_deta_tp = af.tile(dLq_deta_tp, d0 = N_LGL)
dLq_deta_tp = af.reorder(dLq_deta_tp, d0 = 0, d1 = 2, d2 = 1)

In [36]:
dxi_dx  = af.constant(0, N_LGL**2, len(elements), dtype = af.Dtype.f64)
dxi_dy  = af.constant(0, N_LGL**2, len(elements), dtype = af.Dtype.f64)
deta_dx = af.constant(0, N_LGL**2, len(elements), dtype = af.Dtype.f64)
deta_dy = af.constant(0, N_LGL**2, len(elements), dtype = af.Dtype.f64)

for element_tag, element in enumerate(elements):
    dxi_dx[:, element_tag]  = w2d.dxi_dx(nodes[element][:, 0], nodes[element][:, 1], xi_i, eta_j)
    dxi_dy[:, element_tag]  = w2d.dxi_dy(nodes[element][:, 0], nodes[element][:, 1], xi_i, eta_j)
    deta_dx[:, element_tag] = w2d.deta_dx(nodes[element][:, 0], nodes[element][:, 1], xi_i, eta_j)
    deta_dy[:, element_tag] = w2d.deta_dy(nodes[element][:, 0], nodes[element][:, 1], xi_i, eta_j)

In [37]:
def F_xi(u, dxi_dx, dxi_dy):
    '''
    '''
    F_xi_u = w2d.F_x(u) * dxi_dx + w2d.F_y(u) * dxi_dy

    return F_xi_u

def F_eta(u, deta_dx, deta_dy):
    '''
    '''
    F_eta_u = w2d.F_x(u) * deta_dx + w2d.F_y(u) * deta_dy
    
    return F_eta_u

In [38]:
def g_dd(x_nodes, y_nodes, xi, eta):
    '''
    '''
    ans00  =   (w2d.dx_dxi(x_nodes, xi, eta))**2 \
             + (w2d.dy_dxi(y_nodes, xi, eta))**2
    ans11  =   (w2d.dx_deta(x_nodes, xi, eta))**2 \
             + (w2d.dy_deta(y_nodes, xi, eta))**2
    
    ans01  =  (w2d.dx_dxi(x_nodes, xi, eta))  \
            * (w2d.dx_deta(x_nodes, xi, eta)) \
            + (w2d.dy_dxi(y_nodes, xi, eta))  \
            * (w2d.dy_deta(y_nodes, xi, eta))
    
    ans =  [[ans00, ans01],
            [ans01, ans11]
           ]
    
    return ans


def g_uu(x_nodes, y_nodes, xi, eta):
    gCov = g_dd(x_nodes, y_nodes, xi, eta)
    
    
    a = gCov[0][0]
    b = gCov[0][1]
    c = gCov[1][0]
    d = gCov[1][1]
    
    det = (a*d - b*c)
    
    ans = [[d / det, -b / det],
           [-c / det, a / det]]
    
    return ans

In [39]:
G_ab = [[af.constant(0, N_LGL * N_LGL, len(elements), dtype = af.Dtype.f64),
         af.constant(0, N_LGL * N_LGL, len(elements), dtype = af.Dtype.f64)],
        [af.constant(0, N_LGL * N_LGL, len(elements), dtype = af.Dtype.f64),
         af.constant(0, N_LGL * N_LGL, len(elements), dtype = af.Dtype.f64)]]

for element_tag, element in enumerate(elements):
    g_ab = g_uu(nodes[element][:, 0], nodes[element][:, 1], xi_i, eta_j)
    G_ab[0][0][:, element_tag] = g_ab[0][0]
    G_ab[0][1][:, element_tag] = g_ab[0][1]
    G_ab[1][0][:, element_tag] = g_ab[1][0]
    G_ab[1][1][:, element_tag] = g_ab[1][1]

In [40]:
f_xi  = F_xi(u_e_ij, dxi_dx, dxi_dy)
f_eta = F_eta(u_e_ij, deta_dx, deta_dy)

In [65]:
# for p in np.arange(N_LGL):
#     for q in np.arange(N_LGL):
p = 2
q = 6
index = p * N_LGL + q

print(index)
dLp_dxi_tp_ij = utils.polyval_1d(dLp_dxi_tp[index], af.np_to_af_array(np.array(xi_i)))
dLp_dxi_tp_ij = af.tile(af.transpose(dLp_dxi_tp_ij), d0 = 1, d1 = len(elements))

Lq_eta_tp_ij = utils.polyval_1d(Lq_eta_tp[index], af.np_to_af_array(np.array(eta_j)))
Lq_eta_tp_ij = af.tile(af.transpose(Lq_eta_tp_ij), d0 = 1, d1 = len(elements))

Lp_xi_tp_ij = utils.polyval_1d(Lp_xi_tp[index], af.np_to_af_array(np.array(xi_i)))
Lp_xi_tp_ij = af.tile(af.transpose(Lp_xi_tp_ij), d0 = 1, d1 = len(elements))

dLq_deta_tp_ij = utils.polyval_1d(dLq_deta_tp[index], af.np_to_af_array(np.array(eta_j)))
dLq_deta_tp_ij = af.tile(af.transpose(dLq_deta_tp_ij), d0 = 1, d1 = len(elements))


volume_integral_pq_ij = G_ab[0][0] * f_xi  * Lq_eta_tp_ij * dLp_dxi_tp_ij \
                      + G_ab[0][1] * f_eta * Lp_xi_tp_ij * dLq_deta_tp_ij \
                      + G_ab[1][0] * f_xi  * Lq_eta_tp_ij * dLp_dxi_tp_ij \
                      + G_ab[1][1] * f_eta * Lp_xi_tp_ij * dLq_deta_tp_ij

volume_integrand_interpolate = w2d.lag_interpolation_2d(volume_integral_pq_ij,
                                                        N_LGL)
volume_integral_pq = utils.integrate_2d_multivar_poly(volume_integrand_interpolate,
                                                      N_quad = 9, scheme = 'gauss')

22


/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [66]:
af.display(volume_integral_pq[0], 14)

In [45]:
print(nodes[elements[0]])

[[-1.   1. ]
 [-1.   0.9]
 [-1.   0.8]
 [-0.9  0.8]
 [-0.8  0.8]
 [-0.8  0.9]
 [-0.8  1. ]
 [-0.9  1. ]
 [-0.9  0.9]]


In [51]:
af.display(lagrange.gaussian_weights(9), 14)